In [98]:
import pandas as pd

In [99]:
df_objet = pd.read_sql_table('Objet', 'sqlite:///db.sqlite')
df_gare = pd.read_sql_table('Gare', 'sqlite:///db.sqlite')
df_meteo = pd.read_sql_table('Meteo', 'sqlite:///db.sqlite')
df_freq = pd.read_sql_table('Frequence', 'sqlite:///db.sqlite')

In [100]:
df_objet['date'] = pd.to_datetime(df_objet['date']).dt.normalize()

In [101]:
df_objet.columns

Index(['recordid', 'gare_code_uic', 'type', 'nature', 'recordtype', 'date'], dtype='object')

In [102]:
df_freq["code_uic_complet"] = df_freq['code_uic_complet'].apply(lambda x: "00" + str(x))

In [103]:
df_objet_gare = pd.merge(left=df_objet, right=df_gare, how="inner", left_on="gare_code_uic", right_on="uic_code")
df_objet_gare = df_objet_gare.drop('gare_code_uic', axis=1)

In [104]:
df_objet_gare.set_index('date', inplace=True)

In [105]:
df_gare_obj = df_objet_gare.groupby(['uic_code', pd.Grouper(freq = 'Y')]).agg(nb_objets = ('type' , 'count')).reset_index()

In [106]:
df_gare_obj.set_index('uic_code', inplace=True)

In [107]:
nb_obj_y_g = pd.pivot_table(df_gare_obj, index='uic_code', values='nb_objets', columns=df_gare_obj['date'].dt.year)

In [108]:
nb_obj_y_g = nb_obj_y_g.reset_index()

In [109]:
nb_obj_y_g.columns.name = None
nb_obj_y_g = nb_obj_y_g.rename(columns={2016: "obj_2016", 2017: "obj_2017",2018: "obj_2018", 2019: "obj_2019",2020: "obj_2020", 2021: "obj_2021"})
nb_obj_y_g.head()

,uic_code,obj_2016,obj_2017,obj_2018,obj_2019,obj_2020,obj_2021
0,0087111849,644.0,553.0,320.0,291.0,99.0,165.0
1,0087113001,2336.0,1263.0,1376.0,1589.0,844.0,980.0
2,0087141002,959.0,837.0,795.0,701.0,409.0,499.0
3,0087141150,28.0,42.0,40.0,26.0,12.0,7.0
4,0087144006,81.0,141.0,79.0,88.0,29.0,33.0


In [110]:
df_freq_gare = pd.merge(left=df_freq, right=df_gare, how="inner", left_on="code_uic_complet", right_on="uic_code")

In [111]:
df_object_freq = pd.merge(left=nb_obj_y_g, right=df_freq_gare, how="inner", left_on="uic_code", right_on="uic_code")

In [115]:
import fonctions
df_object_freq['gare_regionsncf_libelle'] = df_object_freq['region'].apply(lambda x: fonctions.search_region(x))

In [131]:
df_object_freq

,uic_code,obj_2016,obj_2017,obj_2018,obj_2019,obj_2020,obj_2021,recordid,total_voyageurs_2016,total_voyageurs_2017,...,nom_gare,region,region_name,departement_libellemin,gare_regionsncf_libelle,gare_alias_libelle_noncontraint,adresse_cp,code_gare,lat,long
0,0087111849,644.0,553.0,320.0,291.0,99.0,165.0,43a30fde000a76d44b239948432a8453b2cc7322,5418784,6009276,...,Marne-la-Vallée Chessy,77,Île-de-France,Seine-et-Marne,Île-de-France,Marne-la-Vallée Chessy,77111,03072,48.869856,2.782720
1,0087113001,2336.0,1263.0,1376.0,1589.0,844.0,980.0,35c41e8b83f53818a4f38772aff516c08a0404ce,35820125,38455354,...,Paris Est,75,Île-de-France,Paris,Île-de-France,Paris Est,75010,03070,48.876742,2.358424
2,0087141002,959.0,837.0,795.0,701.0,409.0,499.0,4f3fd5a94f59f5bb32ba3c3d267228b5197eeeb7,7633063,7891228,...,Nancy,54,Grand Est,Meurthe-et-Moselle,Grand Est,Nancy,54000,03002,48.689861,6.174583
3,0087141150,28.0,42.0,40.0,26.0,12.0,7.0,1502a02aed2e96108b038f636a0522c56254c360,877566,918474,...,Lunéville,54,Grand Est,Meurthe-et-Moselle,Grand Est,Lunéville,54300,02988,48.588233,6.497270
4,0087144006,81.0,141.0,79.0,88.0,29.0,33.0,ed9e2ad4c7d3912b9c90b56c4bf698bce3aeb801,1081484,1105348,...,Épinal,88,Grand Est,Vosges,Grand Est,Épinal,88000,02945,48.178005,6.441787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0087773002,2524.0,2293.0,1796.0,1618.0,813.0,1027.0,d8198616cecf33aa5e33153a2c7730ee3667b05d,8131336,8667721,...,Montpellier Saint-Roch,34,Occitanie,Hérault,Occitanie,Montpellier Saint-Roch,34011,00130,43.604738,3.880674
123,0087775007,222.0,320.0,268.0,271.0,197.0,365.0,33e2001044f8999d46aa45d168512dcc293413b1,3937669,4105151,...,Nîmes,30,Occitanie,Gard,Occitanie,Nîmes,30000,00118,43.832291,4.365845
124,0087781005,270.0,246.0,260.0,191.0,71.0,120.0,287f367822516a736e7dd550b32b85bb27eeab4c,1374338,1441893,...,Béziers,34,Occitanie,Hérault,Occitanie,Béziers,34500,00080,43.336330,3.218873
125,0087781104,219.0,210.0,203.0,209.0,186.0,256.0,5c847de4e3f4243f1d20d363fc7156c951f7d124,1508061,1498171,...,Narbonne,11,Occitanie,Aude,Occitanie,Narbonne,11100,00077,43.190387,3.005910
